# Module Title:	 Machine Learning for Business  
### Assessment Title:  MLBus_HDipData_CA1
### Lecturer Name:	 Dr. Muhammad Iqba  
### Student Full Name & Number:	Natalia de Oliveira Rodrigues 2023112 and Heitor Gomes de Araujo Filho 2023098

This CA will assess student attainment of the following minimum intended learning outcomes:

1. Critically evaluate and implement appropriate clustering algorithms and interpret and document 
their results. (Linked to PLO 1, PLO 5)
2. Apply modelling to time series data to facilitate business intelligence needs (Linked to PLO 1, PLO 2, 
PLO 3

**Project Objective:** 
Perform time series analysis on the historical plane crash data and use clustering techniques to identify patterns and clusters of crash incidents over time. 

1. **Temporal Patterns Analysis:** How the frequency of plane crashes has evolved over the years. Are there any long-term trends or seasonal patterns in crash occurrences?

2. **Clustering of Crash Incidents:** Identify commonalities among different incidents using clustering algorithms to group similar plane crashes based on characteristics such as crash causes, flight phases, and other relevant factors. 

3. **Visualization of Clustered Data:** How certain types of crashes have become more or less prevalent over the years?(identified clusters over time)

4. **Anomaly Detection:** These could be extreme or unusual crash incidents that deviate from the typical patterns.

5. **Forecasting:** Predict the future trend of plane crashes based on historical data using time series forecasting models. (valuable tool for aviation safety assessment)

6. **Interpreting Cluster Characteristics:**  Are there specific conditions or causes that lead to certain types of accidents? Investigate the characteristics and factors that contribute for each cluster of crashes formation.

7. **Evaluation of Clustering Methods:** Compare and evaluate different clustering algorithms to determine which one provides the most meaningful insights into the dataset.

**Aims:** 
- Deeper understanding of the historical plane crash data, 
- Identify recurring patterns, 
- Potentially discover factors that contribute to certain types of accidents. 

# Exploratory Data Analysis

In [1]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('../../data/Plane Crashes.csv')

In [4]:
# To convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
def glimpse(df):
    display(f'There are {df.shape[0]} observations and {df.shape[1]} attributes in this dataset.')
    print("-" * 120)
    display(df.head(3))
    print("-" * 120)
    display(df.tail(3))
    print("-" * 120)
    display(df.describe())
    print("-" * 120)
    display(df.info())
    print("-" * 120)
    display(df.isnull().sum().sort_values(ascending=False))
    
glimpse(df)

'There are 28536 observations and 24 attributes in this dataset.'

------------------------------------------------------------------------------------------------------------------------


,Date,Time,Aircraft,Operator,Registration,Flight phase,Flight type,Survivors,Crash site,Schedule,...,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
0,1918-05-02,NaN,De Havilland DH.4,United States Signal Corps - USSC,AS-32084,Takeoff (climb),Test,No,Airport (less than 10 km from airport),Dayton - Dayton,...,United States of America,North America,2.0,2.0,0.0,0.0,0.0,2,The single engine airplane departed Dayton-McC...,Technical failure
1,1918-06-08,NaN,Handley Page V/1500,Handley Page Aircraft Company Ltd,E4104,Takeoff (climb),Test,Yes,Airport (less than 10 km from airport),Cricklewood - Cricklewood,...,United Kingdom,Europe,6.0,5.0,0.0,0.0,0.0,5,"Assembled at Cricklewood Airfield in May 1918,...",Technical failure
2,1918-06-11,NaN,Avro 504,Royal Air Force - RAF,A8544,Flight,Training,Yes,"Plain, Valley",Abukir - Abukir,...,Egypt,Africa,2.0,1.0,0.0,0.0,0.0,1,The single engine aircraft was completing a lo...,Unknown


------------------------------------------------------------------------------------------------------------------------


,Date,Time,Aircraft,Operator,Registration,Flight phase,Flight type,Survivors,Crash site,Schedule,...,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
28533,2022-05-24,15H 40M 0S,De Havilland DHC-3 Otter,Yakutat Coastal Airlines,N703TH,Landing (descent or approach),Charter/Taxi (Non Scheduled Revenue Flight),Yes,Airport (less than 10 km from airport),Yakutat – Dry Bay,...,United States of America,North America,1.0,0.0,3.0,0.0,0.0,0,The single engine airplane departed Yakutat on...,Unknown
28534,2022-05-29,10H 7M 0S,De Havilland DHC-6 Twin Otter,Tara Air,9N-AET,Flight,Scheduled Revenue Flight,No,Mountains,Pokhara – Jomsom,...,Nepal,Asia,3.0,3.0,19.0,19.0,0.0,22,The twin engine airplane departed Pokhara City...,Human factor
28535,2022-06-03,13H 46M 0S,Cessna 208B Grand Caravan,GoJump Oceanside,N7581F,Landing (descent or approach),Skydiving / Paratroopers,Yes,Airport (less than 10 km from airport),Oceanside - Oceanside,...,United States of America,North America,1.0,0.0,1.0,1.0,0.0,1,The single engine was completing local skydivi...,Unknown


...

,Date,YOM,Flight no.,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities
count,28536,23225.000000,0.0,28512.000000,28535.000000,28482.000000,28535.000000,28526.000000,28536.000000
mean,1968-03-30 03:54:02.724978968,1931.942519,NaN,3.052539,1.771649,7.705393,3.679727,0.109760,5.567389
min,1918-05-02 00:00:00,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1945-05-04 18:00:00,1944.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1967-04-29 00:00:00,1958.000000,NaN,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1988-09-20 00:00:00,1974.000000,NaN,4.000000,3.000000,4.000000,1.000000,0.000000,5.000000
max,2022-06-03 00:00:00,19567.000000,NaN,1924.000000,25.000000,509.000000,506.000000,297.000000,520.000000
std,NaN,285.486067,NaN,11.738151,2.520554,24.066368,15.288171,2.644296,16.713203


------------------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28536 entries, 0 to 28535
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              28536 non-null  datetime64[ns]
 1   Time              13949 non-null  object        
 2   Aircraft          28535 non-null  object        
 3   Operator          28536 non-null  object        
 4   Registration      27721 non-null  object        
 5   Flight phase      27898 non-null  object        
 6   Flight type       28479 non-null  object        
 7   Survivors         27239 non-null  object        
 8   Crash site        28153 non-null  object        
 9   Schedule          19590 non-null  object        
 10  MSN               24354 non-null  object        
 11  YOM               23225 non-null  float64       
 12  Flight no

None

------------------------------------------------------------------------------------------------------------------------


Flight no.          28536
Time                14587
Schedule             8946
YOM                  5311
MSN                  4182
Survivors            1297
Registration          815
Flight phase          638
Crash site            383
Flight type            57
Pax on board           54
Circumstances          25
Crew on board          24
Crash location         12
Other fatalities       10
Aircraft                1
Country                 1
Region                  1
Crew fatalities         1
PAX fatalities          1
Total fatalities        0
Date                    0
Operator                0
Crash cause             0
dtype: int64

# Data PreProcessing 

- Delete columns due to high number of missing values: 
Flight no., Time

- Deleted columns with unique values identifier:
MSN, Registration

In [6]:
# This will convert columns dtype
columns_to_convert = ['YOM', 'Crew on board', 'Crew fatalities', 'Pax on board', 'PAX fatalities', 
                      'Other fatalities', 'Total fatalities']

for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

In [7]:
# This removes the 3 attributes mentioned above
df = df.drop(columns=["Flight no.", "Time", 'MSN','Registration'])

In [8]:
# This select recent data the past 10 Years - Allow us to capture contemporary trends and patterns.
df = df[df['Date'] > '31-05-2011']

In [9]:
# Attach your own 'Data' index to the dataframe
df.index = df['Date']

# Drop the 'Date' column from the dataframe
df.drop('Date', axis = 1, inplace = True)

In [10]:
df.isnull().sum().sort_values(ascending=False)

Schedule            151
YOM                  55
Flight type           4
Survivors             2
Flight phase          1
Aircraft              0
Crew fatalities       0
Circumstances         0
Total fatalities      0
Other fatalities      0
PAX fatalities        0
Pax on board          0
Country               0
Crew on board         0
Region                0
Operator              0
Crash location        0
Crash site            0
Crash cause           0
dtype: int64

In [11]:
# This removes all rows with any missing values - Less than 2% of the data will be dropped
df.dropna(inplace=True) 

In [12]:
# Check the data after pre processing data
glimpse(df)

'There are 1441 observations and 19 attributes in this dataset.'

------------------------------------------------------------------------------------------------------------------------


,Aircraft,Operator,Flight phase,Flight type,Survivors,Crash site,Schedule,YOM,Crash location,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
Date,,,,,,,,,,,,,,,,,,,
2011-06-01,Eclipse EA500,James Teng %26 Sons,Landing (descent or approach),Private,Yes,Airport (less than 10 km from airport),Anadyr – Nome,2007,Nome Alaska,United States of America,North America,1,0,1,0,0,0,"The pilot indicated that, prior to the acciden...",Human factor
2011-06-06,Antonov AN-26,Solenta Aviation,Landing (descent or approach),Cargo,Yes,"Lake, Sea, Ocean, River",Port Gentil - Libreville,1978,Libreville Estuaire (Libreville),Gabon,Africa,3,0,1,0,0,0,The crew departed Port Gentil on a cargo fligh...,Technical failure
2011-06-12,Douglas DC-6,Universal Airlines,Landing (descent or approach),Positioning,Yes,Airport (less than 10 km from airport),Togiak - Cold Bay,1956,Cold Bay Alaska,United States of America,North America,4,0,0,0,0,0,"According to the captain, while on approach to...",Human factor


------------------------------------------------------------------------------------------------------------------------


,Aircraft,Operator,Flight phase,Flight type,Survivors,Crash site,Schedule,YOM,Crash location,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
Date,,,,,,,,,,,,,,,,,,,
2022-05-24,De Havilland DHC-3 Otter,Yakutat Coastal Airlines,Landing (descent or approach),Charter/Taxi (Non Scheduled Revenue Flight),Yes,Airport (less than 10 km from airport),Yakutat – Dry Bay,1965,Dry Bay Alaska,United States of America,North America,1,0,3,0,0,0,The single engine airplane departed Yakutat on...,Unknown
2022-05-29,De Havilland DHC-6 Twin Otter,Tara Air,Flight,Scheduled Revenue Flight,No,Mountains,Pokhara – Jomsom,1979,Shikha Dhawalagiri,Nepal,Asia,3,3,19,19,0,22,The twin engine airplane departed Pokhara City...,Human factor
2022-06-03,Cessna 208B Grand Caravan,GoJump Oceanside,Landing (descent or approach),Skydiving / Paratroopers,Yes,Airport (less than 10 km from airport),Oceanside - Oceanside,1994,Oceanside California,United States of America,North America,1,0,1,1,0,1,The single engine was completing local skydivi...,Unknown


...

,YOM,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities
count,1441.0,1441.0,1441.0,1441.0,1441.0,1441.0,1441.0
mean,1981.219292,2.530881,1.111728,14.133935,4.072172,0.125607,5.309507
std,104.675168,2.344745,1.893522,37.455558,19.362702,1.403929,20.640406
min,16.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1977.0,1.0,0.0,0.0,0.0,0.0,0.0
50%,1985.0,2.0,1.0,2.0,0.0,0.0,1.0
75%,1999.0,3.0,1.0,7.0,1.0,0.0,3.0
max,2020.0,18.0,17.0,307.0,283.0,35.0,298.0


------------------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1441 entries, 2011-06-01 to 2022-06-03
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Aircraft          1441 non-null   object
 1   Operator          1441 non-null   object
 2   Flight phase      1441 non-null   object
 3   Flight type       1441 non-null   object
 4   Survivors         1441 non-null   object
 5   Crash site        1441 non-null   object
 6   Schedule          1441 non-null   object
 7   YOM               1441 non-null   Int64 
 8   Crash location    1441 non-null   object
 9   Country           1441 non-null   object
 10  Region            1441 non-null   object
 11  Crew on board     1441 non-null   Int64 
 12  Crew fatalities   1441 non-null   Int64 
 13  Pax on board      1441 non-null   Int64 
 14  PAX fatalitie

None

------------------------------------------------------------------------------------------------------------------------


Aircraft            0
Region              0
Circumstances       0
Total fatalities    0
Other fatalities    0
PAX fatalities      0
Pax on board        0
Crew fatalities     0
Crew on board       0
Country             0
Operator            0
Crash location      0
YOM                 0
Schedule            0
Crash site          0
Survivors           0
Flight type         0
Flight phase        0
Crash cause         0
dtype: int64

In [13]:
# Invalid values column Year of manufacture
df.YOM.unique()

<IntegerArray>
[2007, 1978, 1956, 1944, 1974, 1973, 1980, 1958, 2005, 1965, 1979, 1975, 1977,
 1992, 1995, 2008, 1994, 1989, 1981, 2006, 1997, 1976, 2009, 1963, 1969, 1991,
 2000, 2001, 2010, 1993, 2002, 1968, 2004, 1960, 1990, 1984, 1988, 1996, 1967,
 1970, 1983, 1986, 1972, 1962, 1964, 1999, 1982, 1998, 2003, 1971, 1955, 1953,
 1945, 2012, 2011, 1961, 1966, 1952, 1959, 1987, 1985, 1934, 1957, 2013, 1943,
 1951, 1930,   26, 2014,   18,   23, 2015, 1954, 1928,   16, 2016, 1948, 1939,
 2017, 2018, 1942, 2020]
Length: 82, dtype: Int64

In [14]:
# To create a filter valid_year where only values after 1900 and before 2022 will be kept
valid_years = (df['YOM'] >= 1900) & (df['YOM'] <= 2022)
df = df[valid_years]

In [15]:
glimpse(df)

'There are 1437 observations and 19 attributes in this dataset.'

------------------------------------------------------------------------------------------------------------------------


,Aircraft,Operator,Flight phase,Flight type,Survivors,Crash site,Schedule,YOM,Crash location,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
Date,,,,,,,,,,,,,,,,,,,
2011-06-01,Eclipse EA500,James Teng %26 Sons,Landing (descent or approach),Private,Yes,Airport (less than 10 km from airport),Anadyr – Nome,2007,Nome Alaska,United States of America,North America,1,0,1,0,0,0,"The pilot indicated that, prior to the acciden...",Human factor
2011-06-06,Antonov AN-26,Solenta Aviation,Landing (descent or approach),Cargo,Yes,"Lake, Sea, Ocean, River",Port Gentil - Libreville,1978,Libreville Estuaire (Libreville),Gabon,Africa,3,0,1,0,0,0,The crew departed Port Gentil on a cargo fligh...,Technical failure
2011-06-12,Douglas DC-6,Universal Airlines,Landing (descent or approach),Positioning,Yes,Airport (less than 10 km from airport),Togiak - Cold Bay,1956,Cold Bay Alaska,United States of America,North America,4,0,0,0,0,0,"According to the captain, while on approach to...",Human factor


------------------------------------------------------------------------------------------------------------------------


,Aircraft,Operator,Flight phase,Flight type,Survivors,Crash site,Schedule,YOM,Crash location,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
Date,,,,,,,,,,,,,,,,,,,
2022-05-24,De Havilland DHC-3 Otter,Yakutat Coastal Airlines,Landing (descent or approach),Charter/Taxi (Non Scheduled Revenue Flight),Yes,Airport (less than 10 km from airport),Yakutat – Dry Bay,1965,Dry Bay Alaska,United States of America,North America,1,0,3,0,0,0,The single engine airplane departed Yakutat on...,Unknown
2022-05-29,De Havilland DHC-6 Twin Otter,Tara Air,Flight,Scheduled Revenue Flight,No,Mountains,Pokhara – Jomsom,1979,Shikha Dhawalagiri,Nepal,Asia,3,3,19,19,0,22,The twin engine airplane departed Pokhara City...,Human factor
2022-06-03,Cessna 208B Grand Caravan,GoJump Oceanside,Landing (descent or approach),Skydiving / Paratroopers,Yes,Airport (less than 10 km from airport),Oceanside - Oceanside,1994,Oceanside California,United States of America,North America,1,0,1,1,0,1,The single engine was completing local skydivi...,Unknown


...

,YOM,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities
count,1437.0,1437.0,1437.0,1437.0,1437.0,1437.0,1437.0
mean,1986.676409,2.531663,1.113431,14.146834,4.082116,0.125957,5.321503
std,15.8621,2.347554,1.895331,37.502999,19.388678,1.405868,20.667676
min,1928.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1977.0,1.0,0.0,0.0,0.0,0.0,0.0
50%,1985.0,2.0,1.0,2.0,0.0,0.0,1.0
75%,1999.0,3.0,1.0,7.0,1.0,0.0,3.0
max,2020.0,18.0,17.0,307.0,283.0,35.0,298.0


------------------------------------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1437 entries, 2011-06-01 to 2022-06-03
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Aircraft          1437 non-null   object
 1   Operator          1437 non-null   object
 2   Flight phase      1437 non-null   object
 3   Flight type       1437 non-null   object
 4   Survivors         1437 non-null   object
 5   Crash site        1437 non-null   object
 6   Schedule          1437 non-null   object
 7   YOM               1437 non-null   Int64 
 8   Crash location    1437 non-null   object
 9   Country           1437 non-null   object
 10  Region            1437 non-null   object
 11  Crew on board     1437 non-null   Int64 
 12  Crew fatalities   1437 non-null   Int64 
 13  Pax on board      1437 non-null   Int64 
 14  PAX fatalitie

None

------------------------------------------------------------------------------------------------------------------------


Aircraft            0
Region              0
Circumstances       0
Total fatalities    0
Other fatalities    0
PAX fatalities      0
Pax on board        0
Crew fatalities     0
Crew on board       0
Country             0
Operator            0
Crash location      0
YOM                 0
Schedule            0
Crash site          0
Survivors           0
Flight type         0
Flight phase        0
Crash cause         0
dtype: int64

**Done:**
- Date columns had dtype changed to datatime
- Data must be transformed in the dataset index and ascending sorted
- 4 Columns dropped
- The past 10 years was selected to captured conteponrary trends and patterns (Dataset contain data up to 03-06-2022)
- Less than 2% of missing data was dropped
- YOM, Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities, must be transformed in integer
- YOM has incorrect values like 16,18,23,26 when we are expecting 4 digit value YEAR like 2023

**To do:** Investigate Categorical Data:
- Survivors transfor in numerical to apply ML
- Crash site transfor in numerical to apply ML

In [16]:
categorical_columns = df.select_dtypes(include=['object', 'category'])

In [17]:
categorical_columns.nunique().sort_values(ascending=False)

Circumstances     1435
Schedule          1397
Crash location    1266
Operator          1177
Aircraft           259
Country            133
Flight type         27
Region               9
Crash site           6
Crash cause          6
Flight phase         5
Survivors            2
dtype: int64